In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from nbody import Particles, NbodySimulation

# Sun-Earth System

In this notebook, we will test our Solar System (Sun + Earth) simulation.\
For convenice, that's define the `problem_name` here for handling data IO.

In [2]:
problem_name = "solar_earth"

Prepare physical constants

In [3]:
msun   = 1.989e33   # gram
mearth = 5.97219e27 # gram
au     = 1.496e13   # cm
day    = 86400      # sec
year   = 365*day    # sec
G      = 6.67e-8   # cgs

Re-implment the particle initialze condition of the Sun+Earth system. 

In [4]:
def initialSolarSystem(particles:Particles):
    """
    initial Sun-Earth system
    """
    particles.positions = np.array([[0,0,0], [au,0,0]])
    particles.velocities = np.array([[0,0,0], [0,2*np.pi*au/(year),0]])
    particles.masses = np.array([[msun], [mearth]])
    force = G*msun*mearth/au**2
    particles.accelerations = np.array([[force/msun,0,0], [-force/mearth,0,0]])
    return particles

Once we initialize the particles, we could run our simulation by

In [ ]:
particles = initialSolarSystem(particles=Particles(N=2))
sim = NbodySimulation(particles)
sim.setup(G=G,method="RK4",io_freq=20,io_title=problem_name,io_screen=False,visualized=False)
sim.evolve(dt=0.1*day,tmax=5*year)

In [5]:
particles = initialSolarSystem(particles=Particles(N=2))
sim = NbodySimulation(particles)
sim.setup(G=G,method="RK4",io_freq=-1,io_title=problem_name,io_screen=False,visualized=False)
%timeit sim.evolve(dt=0.1*day,tmax=5*year)

Done!
Done!
Done!
Done!
Done!
Done!
Done!
Done!
378 µs ± 57.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Load data and Visualization

note: `conda install -c conda-forge ffmpeg` might be necessary

In [ ]:
import glob

In [ ]:
fns = "data_"+problem_name+"/"+"[0-9][0-9][0-9][0-9][0-9].txt"
fns = glob.glob(fns)
fns.sort()
#print(fns) 

In [ ]:
scale = 2 * au

fig, ax =plt.subplots()
fig.set_size_inches(10.5, 10.5, forward=True)
fig.set_dpi(72)
ol,   = ax.plot([0,au],[0,0],'ro',alpha=0.3) # the initial conditions
line, = ax.plot([],[],'o')                   # plots of particles

def init():
    ax.set_xlim(-scale,scale)
    ax.set_ylim(-scale,scale)
    ax.set_aspect('equal')
    ax.set_xlabel('X [code unit]')
    ax.set_ylabel('Y [code unit]')
    return line,

def updateParticles(frame):
    fn = fns[frame]
    t,m,x,y,z,vx,vy,vz,ax,ay,az = np.loadtxt(fn)
    print("loadtxt done",fn)
    line.set_data(x,y)
    return line,

ani = animation.FuncAnimation(fig, updateParticles, frames=len(fns),init_func=init, blit=True)
ani.save('movie_'+problem_name+'.mp4',fps=10)